# Data Wrangling With MongoDB Project

**Project 2 from Udacity Data Analyst Nano Degree program              by Jay Zmudka**

---

In [53]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET
from pprint import pprint as pp
import re
import codecs
import json
from zipfile import ZipFile


In [85]:
samplefile = 'sample.osm'
zipfile = 'openstreetmapdata.zip'
filename = 'openstreetmapdata'
outfile = 'openstreetmapdata.json'

# uncomment the data you are using smaller sample file or larger zipped file
usefile = samplefile
##usefile = openzipfile(zipfile, filename)

CREATED = [ 'version', 'changeset', 'timestamp', 'user', 'uid']
ROOTTAGS = [ 'node', 'way', 'relation']

problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

# extracts osm file from zip archive
def openzipfile(zipf, filename):
    with ZipFile(zipf) as osmzip:
        xfile = osmzip.open(filename)
        return xfile

        


In [86]:
def shape_element(element):
    node = {}
    if element.tag in ROOTTAGS:
        node['type'] = element.tag
        node['id'] = element.attrib['id']
        if 'visible' in element.attrib:
            node['visible'] = element.attrib['visible']
        node['created'] = {}
        for x in CREATED:
            node['created'][x] = element.attrib[x]
        if 'lat' in element.attrib:
            node['pos'] = [float(element.attrib['lat']), float(element.attrib['lon'])]
        if len(element.findall('tag')) > 0:
            node['address'] = {}
            for tag in element.findall('tag'):
#                   print tag.attrib
                
                if problemchars.search(tag.attrib['k']):
                    pass
                elif tag.attrib['k'] == 'type':
                    node['relation_type'] = tag.attrib['v']
                elif 'addr:' in tag.attrib['k']:
                    if len(tag.attrib['k'].split(':')) > 2:
                        pass
                    else:
                        addc = tag.attrib['k'].split(':')[1]
                        node['address'][addc] = tag.attrib['v']
                else:
                    node[tag.attrib['k']] = tag.attrib['v']
        if len(element.findall('nd')) > 0:
            node['node_refs'] = []
            for nd in element.findall('nd'):
                node['node_refs'].append(nd.attrib['ref'])

        
        return node
    else:
        return None

    

In [120]:
#auditing functions

def list_totals(usefile):
    tagslist ={}
    total = 0
    for _, element in ET.iterparse(usefile):
        total += 1
        if element.tag not in tagslist:
            tagslist[element.tag] = 1
        else:
            tagslist[element.tag] = tagslist[element.tag] + 1
    print 'Counting all the tags: \n'
    print total, 'tags in total \n'
    pp(tagslist)

def list_websites(usefile):
    websites = []
    for _, element in ET.iterparse(usefile):
        if len(element.findall('tag')) > 0:
            for tag in element.findall('tag'):
                if tag.attrib['k'] == 'website':
                    websites.append(tag.attrib['v'])
    print 'List of websites'
    pp(websites)
    badurls = []
    for w in websites:
        if not w.startswith('http'):
            badurls.append(w)
    print "\nThese websites are not proper URL's" 
    pp(badurls)
    
#def find_tiger(usefile):
tiger_tags = 0
total_ways = 0
for _, element in ET.iterparse(usefile):
    if element.tag == 'way':
        total_ways += 1
        if len(element.findall('tag')) > 0:
            for tag in element.findall('tag'):
                if tag.attrib['k'].startswith('tiger'):
                    tiger_tags += 1
            pass
                
print total_ways
print tiger_tags
                
                    
                

        
            


1362
987


In [100]:
# read data into elementtree objects
# writes out json file



data = []
with codecs.open(outfile, "w") as fo:
    for _, element in ET.iterparse(usefile):
        piece = shape_element(element)
        if piece != None:
            data.append(piece)
            fo.write(json.dumps(piece) + "\n")

#for x in data:
#    if x['type'] == 'relation':
#       pp(x)

